# An introduction to <i>geocontext</i> 
### Pontus Hennerdal, January 2019

This notebook can be used for importing a CSV-file (<i>points</i>) and calculating the geographical context around each point in that file. The geographical context is the k-nearest neighbours in another CSV-file (<i>popLocations</i>, could also be the same file). If the values of the groups in the populated locations is higher then 1, the <i>geocontext</i> function calculates the proportion of the k-nearest neighbours being a part of this group. If the values is between 0 and 1, the function calculates a weighted average and standard deviation for each point among the k-nearest neighbours.

Using the notion used in <i>Hennerdal & Nielsen (2017) http://dx.doi.org/10.1080/24694452.2016.1261685</i>: If $t(x_{i})$ is defined as the total number of inviduals in the location $x_{i}$, we can express the calculation made by this script as statistics calculated for every set $N_{j,k}$ of locations $x_{i}$ located within the distance $r_{j,k}$ from $x_{j}$.

$$N_{j,k} = \left\{x_{i} \in \mathbb{R}^{2} : | x_{i}-x_{j} | \leq r_{j,k} \right\}$$

When $r_{j,k}$ is the smallest possible but the the following criteria that the sum of individuals in the set of locations $N_{j,k}$ is the same or larger than the value $k$ is met.

$$k \leq \sum_{x_{i} \in N_{j,k}}t(x_{i})$$

This is the same as calculating statistics (e.g. proportion or average and standard deviation) among the k-nearest neighbours for a set of locations.

### Using this Jupyter Notebook
Use this Jupyter Notebook by <b>changing the names or values marked with CHANGE in the <i>Read files section</i></b> and preferable also in the <i>Saving the results as a CSV section</i>.

Run cell by pressing SHIFT and ENTER.

If using this script for a publication. Please refer to it as: <i>P. Hennerdal, geocontext (2019), GitHub repository, https://github.com/PonHen/geocontext</i>

# Importing libraries
This first step imports some external libraries needed to run the functions in this Jupyter Notebook.

In [ ]:
import pandas as pd # This library is used to handle data frames.
import math # This library including mathematical functions.
import numpy as np # This library including more mathematical functions.
import time # This library is used to measure the time to run the script.
import matplotlib.pyplot as plt # This library is used for plotting results.

# Defining the <i>geocontext</i> function

In [ ]:
def geocontext(points,
               popLocations,
               groups,
               populations,
               kValues,
               pointsNorth='North',
               pointsEast='East',
               popLocationsNorth='North',
               popLocationsEast='East'):
    """Calculates the geographical context for each point and return a DataFrame.
    
    This function calculate the geographical context around each point in 
    the pandas DataFrame points. The context is the proportion of the 
    population among the k-nearest neighbours in the pandas DataFrame 
    popLocations that is part of a group, if the values of the groups 
    represents the absoulte number of indivuduals being part of a group.
    However, if the values of the groups represents a value between 0 and 1,
    the context is the weighted average and standard deviation for the
    k-nearest neighbours. The distances is measured as the Euclidean distance.

    Keyword arguments:
    points            -- pandas DataFrame with coordinates
    popLocations      -- pandas DataFrame with coordinates, values and populations
    groups            -- list with strings naming the columns holding the values
    populations       -- list with strings naming the columns holding the populations
    kValues           -- list with integers being the k in the k-nearest neighbours
    pointsNorth       -- string naming of the column (default 'North')
    pointsEast        -- string naming of the column (default 'East')
    popLocationsNorth -- string naming of the column (default 'North')
    popLocationsEast  -- string naming of the column (default 'East')
    """
    
    # Checking the format of the variables.
    if type(groups) != list: 
        raise TypeError('groups need to be a list with column names')
    
    if type(populations) == list:
        if len(populations) == 1:
            multi = False
        else:
            multi = True 
            if len(groups) != len(populations):
                raise IndexError('populations need to have the same length as groups')
    else:
        raise TypeError('populations need to be a column name or a list with column names')
          
    if type(kValues) != list: 
        raise TypeError('kValues need to be a list with k-values')
    
    # Create a list stating for each group if it is True or not that the proportion
    # will be calculated. If False, the average and standard deviation will be calculated.
    prop_List = []
    for group in groups:
        if popLocations[group].max() <= 1:
            prop_List.append(False)
        else:
            prop_List.append(True)
    
    # To make sure the k-values are listed ascending, the list with k-values 
    # is sorted.
    kValues.sort()
    
    # Creating space in the points for radius, total, and group 
    # count for every k-value.
    groupIndex = 0
    for group in groups:
        for kValue in kValues:
            if prop_List[groupIndex]:
                points[f'group_{group}_k{kValue}'] = 0
            else:
                points[f'mean_{group}_k{kValue}'] = 0.0
                points[f'std_{group}_k{kValue}'] = 0.0
            
            if multi:
                points[f'radius_{group}_k{kValue}'] = 0
                points[f'total_{group}_k{kValue}'] = 0
            else:
                points[f'radius_k{kValue}'] = 0
                points[f'total_k{kValue}'] = 0
        groupIndex += 1
    
    # Creating space in the popLocations for distance to a point. 
    # This column will be changed for every point in points.
    popLocations['Distance'] = 0
    
    # For every point in points.
    for pointIndex in range(len(points)):
        # Find the coordinates for the point
        startPointNorth = points[pointsNorth].iloc[pointIndex]
        startPointEast = points[pointsEast].iloc[pointIndex]
        
        # Calculate the Euclidean distance from the point to every populated 
        # location in popLocations.
        popLocations['Distance'] = np.sqrt(
            np.power(startPointNorth - popLocations[popLocationsNorth], 2) 
            + np.power(startPointEast - popLocations[popLocationsEast], 2)).astype('int')
        
        # Sort populated locations by distance to the point.
        popLocations.sort_values('Distance', inplace=True) 
        
        # For every column used to sum up to the k-value
        groupIndex = 0
        for pop_Column in populations:
            
            # Set start values for every point (and category of totals)
            radius_List = []
            population = 0 # This is the value that values will be added to until it is larger than k
            index = 0 # This index will relate to the populated locations sorted from nearest
        
            # Going through every k-value in the list. 
            for kValue in kValues:
                
                # While the poplation is lower than the k-value
                while population < kValue:
                    # Add the population of that populated location
                    population += popLocations[pop_Column].iloc[index]
                    index += 1
            
                # When the population is larger than the k-value: save the 
                # radius to the radius_List.
                radius = popLocations['Distance'].iloc[index - 1]
                radius_List.append(radius)
                if multi:
                    points[f'radius_{groups[groupIndex]}_k{kValue}'].iat[pointIndex] = radius
                else:
                    points[f'radius_k{kValue}'].iat[pointIndex] = radius

            # When the the max radius for all k-values are found: go through 
            # every k-value in the list again.
            for k_Index in range(len(kValues)):
                # For each k-value, select all populated locations upp to 
                # the previous found max distance.
                selection = popLocations[popLocations['Distance'] <= radius_List[k_Index]]
            
                # For each group, calculate the sum of that group among the 
                # selected nearest neighbours.
                if multi:
                    if prop_List[groupIndex]:
                        points[
                            f'group_{groups[groupIndex]}_k{kValues[k_Index]}'
                        ].iat[pointIndex] = selection[groups[groupIndex]].sum()
                    else:
                        average = np.average(selection[groups[groupIndex]], 
                                             weights=selection[pop_Column])
                        variance = np.average((selection[groups[groupIndex]] - average)**2, 
                                              weights=selection[pop_Column])
                        
                        points[
                            f'mean_{groups[groupIndex]}_k{kValues[k_Index]}'
                        ].iat[pointIndex] = average
                        points[
                            f'std_{groups[groupIndex]}_k{kValues[k_Index]}'
                        ].iat[pointIndex] = math.sqrt(variance)
                else:
                    for group in groups:
                        if prop_List[groupIndex]:
                            points[
                                f'group_{group}_k{kValues[k_Index]}'
                            ].iat[pointIndex] = selection[group].sum()
                        else:
                            average = np.average(selection[group], 
                                                 weights=selection[pop_Column])
                            variance = np.average((selection[group] - average)**2, 
                                                  weights=selection[pop_Column])
                            points[
                                f'mean_{group}_k{kValues[k_Index]}'
                            ].iat[pointIndex] = average
                            points[
                                f'std_{group}_k{kValues[k_Index]}'
                            ].iat[pointIndex] = math.sqrt(variance)
            
                # Calculate the sum of selected nearest neighbours
                if multi:
                    points[
                        f'total_{groups[groupIndex]}_k{kValues[k_Index]}'
                    ].iat[pointIndex] = selection[pop_Column].sum() 
                else:
                    points[
                        'total_k'+str(kValues[k_Index])
                    ].iat[pointIndex] = selection[pop_Column].sum() 
            
            groupIndex += 1

    # Calculate the proportion those groups are of the total amount of 
    # neighbours. If statistics=True, also other statistics are 
    # calculated.
    for kValue in kValues:
        groupIndex = 0
        for group in groups:
            if prop_List[groupIndex]:
                if multi:
                    points[f'mean_{group}_k{kValue}'] = points[
                        f'group_{group}_k{kValue}'] / points[f'total_{group}_k{kValue}']
                else:
                    points[f'mean_{group}_k{kValue}'] = points[
                        f'group_{group}_k{kValue}'] / points[f'total_k{kValue}']
            groupIndex += 1
    
    # The function returns points with the added columns.
      
    return points

# Read files (CHANGE)
### Read file with points
The cell below is reading a CSV-file with the points the context will be calculated around and save it as the pandas DataFrame <i>pointsNoContextDF</i>. The file need to be located in the same folder as the Jupyter Notebook and the code in the cell need to be changed to match the name of the CSV-file and the delimiter used in that file.
```python
pointsNoContextDF = pd.read_csv('FILE_NAME.csv', delimiter=',')
```
After running the cell you can see the column names and the first five rows. Find the names of the columns holding the coordinates for North and East respectivly. Change the code in the next cell to match the names of the columns.
```python
points_NorthColumnName = 'NAME_OF_COLUMN_NORTH'
points_EastColumnName = 'NAME_OF_COLUMN_EAST'
```

In [ ]:
# Reads the file. The file should be saved in the same folder 
# as this jupyter notebook.
pointsNoContextDF = pd.read_csv('CHANGE.csv', delimiter='CHANGE')

print(f'There is {len(pointsNoContextDF)} rows, here is the first 5:')
pointsNoContextDF.head()

In [ ]:
pointsNorthName = 'CHANGE'
pointsEastName = 'CHANGE'

### Read file with populated locations
The cell below is reading a CSV-file with the the populated locations that constitutes the context and save it as the pandas DataFrame <i>popLocationsDF</i>. The file need to be located in the same folder as the Jupyter Notebook and the code in the cell need to be changed to match the name of the CSV-file and the delimiter used in that file.
```python
popLocations = pd.read_csv('FILE_NAME_OF_OUTPUT_FILE.csv', delimiter=',')
```

In [ ]:
# Reads the file. The file should be saved in the same folder 
# as this jupyter notebook.
popLocationsDF = pd.read_csv('CHANGE.csv', delimiter='CHANGE')

print(f'There is {len(popLocationsDF)} rows, here is the first 5:')
popLocationsDF.head()

Look att the column names in the file with populated locations that can be seen above after reading the file. Below, list the groups you want to include in the calculation. For example:
```python
groupList = ['NAME_OF_COLUMN_CATEGORY1', 
             'NAME_OF_COLUMN_CATEGORY2', 
             'NAME_OF_COLUMN_CATEGORY3']
```
If there is only one column stating the total population for the populated location, vrite the name as a string in the list <i>populationList</i>
```python
populationList = ['NAME_OF_COLUMN_TOTAL_POPULATION']
```
If the total population differ between different groups (for example if the groups is for different years and the total population differ between different years), then make <i>populationList</i> a list with the column with the same length as <i>groupList</i> and with the column names for the coresponding total populations. 
```python
populationList = ['NAME_OF_COLUMN_TOTAL_POPULATION_CATEGORY1', 
                  'NAME_OF_COLUMN_TOTAL_POPULATION_CATEGORY2', 
                  'NAME_OF_COLUMN_TOTAL_POPULATION_CATEGORY3']
```
Find the names of the columns holding the coordinates for North and East respectivly. Change the code in the next cell to match the names of the columns.
```python
popLocationsNorthName = 'NAME_OF_COLUMN_NORTH_COORDINATE'
popLocationsEastName = 'NAME_OF_COLUMN_EAST_COORDINATE'
```

In [ ]:
groupList = ['CHANGE', 'CHANGE', 'CHANGE']
populationList = ['CHANGE']
#populationList = ['CHANGE', 'CHANGE', 'CHANGE']
popLocationsNorthName = 'CHANGE'
popLocationsEastName = 'CHANGE'

The list of k-values contains the size of the context. If k=1000, statistics will be calculated for the 1000 nearest neighbours for every point in the python DataFrame <i>pointsNoContextDF</i>.
```python
kValueList = [1000, 10000] 
```

In [ ]:
kValueList = [CHANGE, CHANGE]

# The actual calculations
Use the above defined function to calculate geographical context around each point.

In [ ]:
# Start the stopwatch.
time0 = time.time()

# The actual calculation of the geographical context.
pointsWithContext = geocontext(points=pointsNoContextDF,
                               popLocations=popLocationsDF, 
                               groups=groupList, 
                               populations=populationList,
                               kValues=kValueList,
                               pointsNorth=pointsNorthName,
                               pointsEast=pointsEastName,
                               popLocationsNorth=popLocationsNorthName,
                               popLocationsEast=popLocationsEastName)

# Print the time for the calulation.
print(f'{round(time.time() - time0)} seconds')

# Results

### Some statistics

In [ ]:
pointsWithContext.describe(percentiles=[0.5]).transpose()

### The result as a table

In [ ]:
pointsWithContext

### Plotting the result
As default, the plot will show the first group with the largest k-value. 

In [ ]:
plotWidth = 1.5 + (10 
                   * (pointsWithContext[pointsEastName].max() 
                      - pointsWithContext[pointsEastName].min()) 
                   / (pointsWithContext[pointsNorthName].max() 
                      - pointsWithContext[pointsNorthName].min()))

plt.rcParams['figure.figsize'] = [plotWidth, 10]

pointsWithContext.plot.scatter(x=pointsEastName, 
                               y=pointsNorthName, 
                               c=f'mean_{groupList[0]}_k{kValueList[-1]}', 
                               colormap='viridis', s=1);

# Saving the results as a CSV (CHANGE)
The file with the results will be saved as a CSV-file in the same folder as the Jupyter Notebook. Change the name of the file in the code below.
```python
points.to_csv('FILE_NAME_OF_OUTPUT_FILE.csv')
```

In [ ]:
pointsWithContext.to_csv('CHANGE.csv')